In [ ]:
import os
import glob
import subprocess
import shutil
import tempfile
from tqdm import tqdm
import soundfile as sf
import numpy as np
import tensorflow as tf
from spleeter.separator import Separator

# Set variables
singer_folder = r"C:\Users\SSAFY\Desktop\origin_song"
output_folder = r"C:\Users\SSAFY\Desktop\wav_song"

# Set audio settings
sample_rate = 16000
channels = 2


def convert_to_wav(file_path, output_folder):
    print("convert to wav", file_path)
    """Converts an mp3 file to a wav file"""
    # Set the output file path
    filename = os.path.splitext(os.path.basename(file_path))[0]
    output_file = os.path.join(output_folder, f"{filename}.wav")

    # Convert the file using ffmpeg
    subprocess.call(['ffmpeg', '-i', file_path, '-ac', str(channels), '-ar', str(sample_rate), output_file])

    return output_file

def separate_vocals_and_save(file_path, output_folder, singer):
    # Set Spleeter settings
    separator = Separator('spleeter:2stems')
    
    print("separate vocals and save", file_path)
    """Separates the vocals from an audio file and saves them to the output folder"""
    # Load the audio file using soundfile
    audio, _ = sf.read(file_path)
    print(audio.shape)

    # Normalize the audio to the range [-1, 1]
    audio = audio / np.max(np.abs(audio))

    # Set output folder path with singer name
    output_root_folder = os.path.join(output_folder, singer)

    # Separate the vocals using Spleeter
    # Save the vocals to a file
    for instrument, data in separator.separate(audio).items():
        print(data.shape)
        if instrument == 'vocals':
            filename = os.path.splitext(os.path.basename(file_path))[0]
            output_file = os.path.join(output_root_folder, f"{filename}.wav")
            if not os.path.exists(output_root_folder):
                os.makedirs(output_root_folder)
            sf.write(output_file, data, sample_rate, 'PCM_24')

def process_file(file_path, singer):
    # print("process file", file_path, singer)

    """Converts an mp3 file to a wav file, separates the vocals, and saves them to a folder"""
    # Create a temporary directory to store the files
    with tempfile.TemporaryDirectory() as temp_dir:
        # Convert the file to a wav file
        wav_file = convert_to_wav(file_path, temp_dir)
        # print("wav_file 실행 후 seperate 전", file_path, wav_file)
        
        # Separate the vocals and save them to the output folder
        separate_vocals_and_save(wav_file, output_folder, singer)

        # Delete the temporary directory
        shutil.rmtree(temp_dir)

        # Delete the original mp3 file
        # os.remove(file_path)

def main():

    dir_list = os.listdir(singer_folder)
    print(singer_folder, dir_list)

    for dir in dir_list:
        mp3_folder = os.path.join(singer_folder, dir)

        # Get a list of mp3 files in the mp3 folder
        mp3_files = glob.glob(os.path.join(mp3_folder, "*.mp3"))

        # Process each mp3 file
        for mp3_file in tqdm(mp3_files):
            # print("main", mp3_file)
            process_file(mp3_file, dir)

if __name__ == "__main__":
    # Disable tensorflow logging messages
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Run the main function
    main()